In [7]:

import time
from typing import List, Tuple, Dict

import ase.visualize
import numpy as np
from scipy import optimize

import milad
from milad import atomic, invariants, generate, zernike, geometric, functions

import logging

# log = logging.getLogger('milad')
# log.setLevel(logging.DEBUG)
#
# formatter = logging.Formatter('%(message)s')
#
# ch = logging.StreamHandler()
# ch.setLevel(logging.DEBUG)
# ch.setFormatter(formatter)
# log.addHandler(ch)

moment_calc = geometric

# np.random.seed(123)
np.random.seed(12345)


def prepare_molecule(molecule: ase.Atoms) -> Tuple[ase.Atoms, float]:
    com = molecule.get_center_of_mass()
    molecule.set_positions(molecule.positions - com)
    new_positions = molecule.positions
    max_dist_sq = max(np.dot(pos, pos) for pos in new_positions)
    scale_factor = 0.7 / max_dist_sq ** 0.5
    return molecule, scale_factor


# Get the molecule
urea, scale_factor = prepare_molecule(ase.io.read('/home/martin/src/milad/unsaved/urea.pdb'))

NUM_POINTS = len(urea)
READ_MAX = 256

# Get invariants
invariants = milad.invariants.read(filename=invariants.COMPLEX_INVARIANTS, read_max=READ_MAX)
ORDER = invariants.max_order

builder = atomic.AtomsCollectionBuilder(NUM_POINTS)
# builder.numbers = NUMBERS  # Constrain the numbers
# builder.positions = urea.positions

build_inverse = builder.inverse

numbers_range = (0.1, 1.)
preprocess = atomic.MapNumbers(set(urea.numbers), numbers_range)
unprocess = preprocess.inverse

calc = milad.functions.Chain(
    builder,
    atomic.ScalePositions(scale_factor),
    atomic.FeatureMapper(
        # functions.WeightedDelta,
        functions.WeightedGaussian,
        feature_kwargs={'weight': 1.},
        map_species_to=functions.WeightedGaussian.SIGMA
    ),
    milad.zernike.ZernikeMomentCalculator(ORDER),  # Calculate the moments
    # invariants  # Calculate the invariants
)

target_atoms = milad.atomic.AtomsCollection(NUM_POINTS, urea.positions, urea.numbers)

# Calculate the invariants and set fixed values of moments for fitting
phi_target = calc(build_inverse(preprocess(target_atoms)))
# print(phi_target)
# phi_target = np.random.rand(len(invariants))

# Now add the residuals step to the end of the chain
calc.append(milad.functions.Residuals(phi_target))

traj = []
count = [0, time.time()]
track_every = 1


def save_trajectory(input, output: atomic.AtomsCollection, jacobian):
    atoms_collection: atomic.AtomsCollection = unprocess(output)

    if count[0] % track_every == 0:
        traj.append(ase.Atoms(
            positions=atoms_collection.positions,
            numbers=atoms_collection.numbers
        ))


builder.add_callback(save_trajectory)


def step(input, output, jacobian):
    if count[0] % track_every == 0:
        residuals = output
        now = time.time()
#         print("{}: {}, delta t: {}".format(count[0], np.abs(residuals).max(), now - count[1]))
        count[1] = now

    count[0] += 1


calc.add_callback(step)


def calc_wrap(vec: np.ndarray) -> np.ndarray:
    value = calc(vec)
    # return value.real
    return np.concatenate((value.real, value.imag))
    # return value.real
    # res = np.empty(2 * len(value))
    # res[:len(value)] = value.real
    # res[len(value):] = value.imag
    # return res


def jac(vec: np.array) -> np.array:
    _, jacobian = calc(vec, jacobian=True)
    return np.concatenate((jacobian.real, jacobian.imag))
    # return jacobian.real


initial_structure = atomic.AtomsCollection(NUM_POINTS)
initial_structure.positions = generate.random_points_in_sphere(
    NUM_POINTS, radius=0.9 / scale_factor, centre=False)
initial_structure.numbers = urea.numbers[0]

lower_bounds = []
upper_bounds = []
lower_bounds.extend([-1. / scale_factor] * NUM_POINTS * 3)
upper_bounds.extend([1. / scale_factor] * NUM_POINTS * 3)  # Positions can be anywhere
lower_bounds.extend([numbers_range[0]] * NUM_POINTS)  # Weights should be positive
upper_bounds.extend([numbers_range[1]] * NUM_POINTS)  # Weights should be positive

x0 = build_inverse(preprocess(initial_structure))

print("DECODING")
try:
    result = optimize.least_squares(
        calc_wrap,
        x0=x0,
        # jac='3-point',
        # jac=jac,
        bounds=(lower_bounds, upper_bounds),
        xtol=1e-6,
        # x_scale='jac',
    )
except KeyboardInterrupt:
    pass
else:
    print(f"{result.success} {result.cost}")

#ase.visualize.view(ase.Atoms(positions=urea.positions, symbols=urea.symbols))
#ase.visualize.view(traj)

DECODING
True 1.7724986460603157e-20


In [3]:
from ase.io.trajectory import Trajectory
t = Trajectory('urea.traj', 'w', atoms=traj)

In [16]:
from ase import io
for i, s in enumerate(traj):
#     rotate to the desired direction
#     s.rotate('z', 'x', rotate_cell=True)

    ofname = str(i) + '.png'
    print('writing', ofname)
    io.write(ofname, s,
             bbox=[-3, -3, 3, 3], scale=100)  # set bbox by hand, try and error


writing 0.png
writing 1.png
writing 2.png
writing 3.png
writing 4.png
writing 5.png
writing 6.png
writing 7.png
writing 8.png
writing 9.png
writing 10.png
writing 11.png
writing 12.png
writing 13.png
writing 14.png
writing 15.png
writing 16.png
writing 17.png
writing 18.png
writing 19.png
writing 20.png
writing 21.png
writing 22.png
writing 23.png
writing 24.png
writing 25.png
writing 26.png
writing 27.png
writing 28.png
writing 29.png
writing 30.png
writing 31.png
writing 32.png
writing 33.png
writing 34.png
writing 35.png
writing 36.png
writing 37.png
writing 38.png
writing 39.png
writing 40.png
writing 41.png
writing 42.png
writing 43.png
writing 44.png
writing 45.png
writing 46.png
writing 47.png
writing 48.png
writing 49.png
writing 50.png
writing 51.png
writing 52.png
writing 53.png
writing 54.png
writing 55.png
writing 56.png
writing 57.png
writing 58.png
writing 59.png
writing 60.png
writing 61.png
writing 62.png
writing 63.png
writing 64.png
writing 65.png
writing 66.png
writi

writing 523.png
writing 524.png
writing 525.png
writing 526.png
writing 527.png
writing 528.png
writing 529.png
writing 530.png
writing 531.png
writing 532.png
writing 533.png
writing 534.png
writing 535.png
writing 536.png
writing 537.png
writing 538.png
writing 539.png
writing 540.png
writing 541.png
writing 542.png
writing 543.png
writing 544.png
writing 545.png
writing 546.png
writing 547.png
writing 548.png
writing 549.png
writing 550.png
writing 551.png
writing 552.png
writing 553.png
writing 554.png
writing 555.png
writing 556.png
writing 557.png
writing 558.png
writing 559.png
writing 560.png
writing 561.png
writing 562.png
writing 563.png
writing 564.png
writing 565.png
writing 566.png
writing 567.png
writing 568.png
writing 569.png
writing 570.png
writing 571.png
writing 572.png
writing 573.png
writing 574.png
writing 575.png
writing 576.png
writing 577.png
writing 578.png
writing 579.png
writing 580.png
writing 581.png
writing 582.png
writing 583.png
writing 584.png
writing 

writing 1039.png
writing 1040.png
writing 1041.png
writing 1042.png
writing 1043.png
writing 1044.png
writing 1045.png
writing 1046.png
writing 1047.png
writing 1048.png
writing 1049.png
writing 1050.png
writing 1051.png
writing 1052.png
writing 1053.png
writing 1054.png
writing 1055.png
writing 1056.png
writing 1057.png
writing 1058.png
writing 1059.png
writing 1060.png
writing 1061.png
writing 1062.png
writing 1063.png
writing 1064.png
writing 1065.png
writing 1066.png
writing 1067.png
writing 1068.png
writing 1069.png
writing 1070.png
writing 1071.png
writing 1072.png
writing 1073.png
writing 1074.png
writing 1075.png
writing 1076.png
writing 1077.png
writing 1078.png
writing 1079.png
writing 1080.png
writing 1081.png
writing 1082.png
writing 1083.png
writing 1084.png
writing 1085.png
writing 1086.png
writing 1087.png
writing 1088.png
writing 1089.png
writing 1090.png
writing 1091.png
writing 1092.png
writing 1093.png
writing 1094.png
writing 1095.png
writing 1096.png
writing 1097.p

writing 1525.png
writing 1526.png
writing 1527.png
writing 1528.png
writing 1529.png
writing 1530.png
writing 1531.png
writing 1532.png
writing 1533.png
writing 1534.png
writing 1535.png
writing 1536.png
writing 1537.png
writing 1538.png
writing 1539.png
writing 1540.png
writing 1541.png
writing 1542.png
writing 1543.png
writing 1544.png
writing 1545.png
writing 1546.png
writing 1547.png
writing 1548.png
writing 1549.png
writing 1550.png
writing 1551.png
writing 1552.png
writing 1553.png
writing 1554.png
writing 1555.png
writing 1556.png
writing 1557.png
writing 1558.png
writing 1559.png
writing 1560.png
writing 1561.png
writing 1562.png
writing 1563.png
writing 1564.png
writing 1565.png
writing 1566.png
writing 1567.png
writing 1568.png
writing 1569.png
writing 1570.png
writing 1571.png
writing 1572.png
writing 1573.png
writing 1574.png
writing 1575.png
writing 1576.png
writing 1577.png
writing 1578.png
writing 1579.png
writing 1580.png
writing 1581.png
writing 1582.png
writing 1583.p

writing 2015.png
writing 2016.png
writing 2017.png
writing 2018.png
writing 2019.png
writing 2020.png
writing 2021.png
writing 2022.png
writing 2023.png
writing 2024.png
writing 2025.png
writing 2026.png
writing 2027.png
writing 2028.png
writing 2029.png
writing 2030.png
writing 2031.png
writing 2032.png
writing 2033.png
writing 2034.png
writing 2035.png
writing 2036.png
writing 2037.png
writing 2038.png
writing 2039.png
writing 2040.png
writing 2041.png
writing 2042.png
writing 2043.png
writing 2044.png
writing 2045.png
writing 2046.png
writing 2047.png
writing 2048.png
writing 2049.png
writing 2050.png
writing 2051.png
writing 2052.png
writing 2053.png
writing 2054.png
writing 2055.png
writing 2056.png
writing 2057.png
writing 2058.png
writing 2059.png
writing 2060.png
writing 2061.png
writing 2062.png
writing 2063.png
writing 2064.png
writing 2065.png
writing 2066.png
writing 2067.png
writing 2068.png
writing 2069.png
writing 2070.png
writing 2071.png
writing 2072.png
writing 2073.p